In [4]:
import sys
sys.path.append('..')
import torch
from torchmetrics.classification import BinaryF1Score, F1Score
import wandb
from tqdm import trange
import os
from datetime import datetime
from sklearn import metrics
from matplotlib import pyplot as plt
import torch
import torch.nn as nn
import pytorch_lightning as pl
import numpy as np
import json

# sys.path.insert(0, '/Users/evanpan/Documents/GitHub/EvansToolBox/Utils')
# sys.path.insert(0, '/Users/evanpan/Desktop/openpose/python/')
# sys.path.insert(0, '/scratch/ondemand27/evanpan/Gaze_project/')

# from training.model import *
from dataloader import *

/usr/lib/python3/dist-packages/pkg_resources/__init__.py:116: PkgResourcesDeprecationWarning: 0.1.43ubuntu1 is an invalid version and will not be supported in a future release
  warnings.warn(
/usr/lib/python3/dist-packages/pkg_resources/__init__.py:116: PkgResourcesDeprecationWarning: 1.1build1 is an invalid version and will not be supported in a future release
  warnings.warn(
/usr/lib/python3/dist-packages/pkg_resources/__init__.py:116: PkgResourcesDeprecationWarning: 0.1.43ubuntu1 is an invalid version and will not be supported in a future release
  warnings.warn(
/usr/lib/python3/dist-packages/pkg_resources/__init__.py:116: PkgResourcesDeprecationWarning: 1.1build1 is an invalid version and will not be supported in a future release
  warnings.warn(


In [5]:
%load_ext autoreload
%autoreload 1

%aimport training.model
%aimport Dataset_Util.dataloader

ModuleNotFoundError: No module named 'training'

# Loaded Things

In [12]:
dataset_location = "output"
model_save_location = "models"
config = json.load(open("sentence_config.json", "r"))

## Load dataset

In [11]:
# do the training test split here:
dataset_metadata = "output/video_to_window_metadata.json"
dataset_metadata = json.load(open(dataset_metadata, "r"))
all_videos = list(dataset_metadata.keys())
training_set = []
testing_set = []
# get the name of the videos (this ensures no contamination because the same shot is split)
for i in range(0, len(all_videos)):
    if i / len(all_videos) < 0.9:
        training_set.append(all_videos[i])
    else:
        testing_set.append(all_videos[i])

In [8]:
class SentenceBaseline_GazePredictionModel(nn.Module):
    def __init__(self, config):
        torch.set_default_tensor_type(torch.DoubleTensor)
        # initialize model
        super(SentenceBaseline_GazePredictionModel, self).__init__()
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.activation = nn.Sigmoid()
        self.num_layers = config["num_layers"]
        self.config = config
        # the feature of each speaker are encoded with a separate Linear Layer
        self.input_layer_self = nn.Linear(int(config["input_dims"]/2 - 6), config["input_layer_out"])
        self.input_layer_other = nn.Linear(int(config["input_dims"]/2 - 6), config["input_layer_out"])
        
        # the Recurrent Layer will take care of the next step
        self.lstm_hidden_dims = config["lstm_output_feature_size"]
        self.num_lstm_layer = config["lstm_layer_num"]
        self.frames_ahead = config["frames_ahead"]
        self.frames_behind = config["frames_behind"]
        self.lstm = nn.LSTM(2 * (config["input_layer_out"] + 6) * (self.frames_ahead + self.frames_behind + 1), 
                            self.lstm_hidden_dims, 
                            self.num_lstm_layer, 
                            batch_first=True)        
        # output layers
        self.output_layer_1 = nn.Linear(self.lstm_hidden_dims, config["output_layer_1_hidden"])
        self.output_layer_1 = nn.Sequential(self.output_layer_1, self.activation, nn.Dropout(self.config["dropout"]))
        self.output_layer_2 = nn.Linear(config["output_layer_1_hidden"], config["output_layer_2_hidden"])
        self.output_layer_2 = nn.Sequential(self.output_layer_2, self.activation, nn.Dropout(self.config["dropout"]))
        self.output_layer_3 = nn.Linear(config["output_layer_2_hidden"], config["output_layer_3_hidden"])
        self.output_layer_3 = nn.Sequential(self.output_layer_3)

        # audio_filler = torch.tensor([[[-36.04365338911715,0.0,0.0,0.0,0.0,0.0,-3.432169450445466e-14,0.0,0.0,0.0,9.64028691651994e-15,0.0,0.0,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715]]]).to(self.device)
        # text_filler = torch.ones([1, 1, 772]).to(self.device) * -15
        # text_filler[:, :, -4:] = 0
        # self.filler = torch.concat([audio_filler, text_filler], axis=2)
    def concate_frames(self, input_feature):
        # here I expect the 
        padding_front = torch.zeros([input_feature.shape[0], self.frames_ahead, input_feature.shape[2]]).to(self.device)
        padding_back = torch.zeros([input_feature.shape[0], self.frames_behind, input_feature.shape[2]]).to(self.device)
        padded_input_audio = torch.cat([padding_front, input_feature, padding_back], dim=1)
        window_audio = []
        for i in range(0, input_feature.shape[1]):
            window_count = i + self.frames_ahead
            current_window = padded_input_audio[:, window_count-self.frames_ahead:window_count+self.frames_behind+1]
            s = current_window.shape
            current_window = current_window.view((s[0], s[1] * s[2]))
            current_window = torch.unsqueeze(current_window, 1)
            window_audio.append(current_window)
        rtv = torch.cat(window_audio, dim=1)
        return rtv
    def forward(self, input_feature):
        feature_size = int(input_feature.size()[2] / 2)
        mod_audio_self = input_feature[:, :, :feature_size]
        mod_audio_other = input_feature[:, :, feature_size:]
        
        text_feature_self = mod_audio_self[:, :, :6]
        mod_audio_self = mod_audio_self[:, :, 6:]
        text_feature_other = mod_audio_self[:, :, :6]
        mod_audio_other = mod_audio_other[:, :, 6:]
        x1 = self.activation(self.input_layer_self(mod_audio_self))
        x2 = self.activation(self.input_layer_self(mod_audio_other))
        x1_windowed = self.concate_frames(x1)
        x2_windowed = self.concate_frames(x2)
        x_combined = torch.concat([x1_windowed, text_feature_self, x2_windowed, text_feature_other], axis=2)
        # here I'm assuming that the input_audio is of proper shape
        outtruef.load_state_dict(model_dict)

In [9]:
class SimpleBaseline_GazePredictionModel(nn.Module):
    def __init__(self, config):
        torch.set_default_tensor_type(torch.DoubleTensor)
        # initialize model
        super(SimpleBaseline_GazePredictionModel, self).__init__()
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.activation = nn.Sigmoid()
        self.num_layers = config["num_layers"]
        self.config = config
        # the feature of each speaker are encoded with a separate Linear Layer
        self.input_layer_self = nn.Linear(int(config["input_dims"]/2), config["input_layer_out"])
        self.input_layer_other = nn.Linear(int(config["input_dims"]/2), config["input_layer_out"])
        
        # ttruehe Recurrent Layer will take care of the next step
        self.lstm_hidden_dims = config["lstm_output_feature_size"]
        self.num_lstm_layer = config["lstm_layer_num"]
        self.frames_ahead = config["frames_ahead"]
        self.frames_behind = config["frames_behind"]
        self.lstm = nn.LSTM(2 * config["input_layer_out"] * (self.frames_ahead + self.frames_behind + 1), 
                            self.lstm_hidden_dims, 
                            self.num_lstm_layer, 
                            batch_first=True)
        self.lstm = nn.LSTM(2 * config["input_layer_out"] * (self.frames_ahead + self.frames_behind + 1), 
                            self.lstm_hidden_dims, 
                            self.num_lstm_layer, 
                            batch_first=True)
        
        # output layers
        self.output_layer_1 = nn.Linear(self.lstm_hidden_dims, config["output_layer_1_hidden"])
        self.output_layer_1 = nn.Sequential(self.output_layer_1, self.activation, nn.Dropout(self.config["dropout"]))
        self.output_layer_2 = nn.Linear(config["output_layer_1_hidden"], config["output_layer_2_hidden"])
        self.output_layer_2 = nn.Sequential(self.output_layer_2, self.activation, nn.Dropout(self.config["dropout"]))
        self.output_layer_3 = nn.Linear(config["output_layer_2_hidden"], config["output_layer_3_hidden"])
        self.output_layer_3 = nn.Sequential(self.output_layer_3, nn.Sigmoid())

        # audio_filler = torch.tensor([[[-36.04365338911715,0.0,0.0,0.0,0.0,0.0,-3.432169450445466e-14,0.0,0.0,0.0,9.64028691651994e-15,0.0,0.0,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715]]]).to(self.device)
        # text_filler = torch.ones([1, 1, 772]).to(self.device) * -15
        # text_filler[:, :, -4:] = 0
        # self.filler = torch.concat([audio_filler, text_filler], axis=2)
    def concate_frames(self, input_feature):
        # here I expect the 
        padding_front = torch.zeros([input_feature.shape[0], self.frames_ahead, input_feature.shape[2]]).to(self.device)
        padding_back = torch.zeros([input_feature.shape[0], self.frames_behind, input_feature.shape[2]]).to(self.device)
        padded_input_audio = torch.cat([padding_front, input_feature, padding_back], dim=1)
        window_audio = []
        for i in range(0, input_feature.shape[1]):
            window_count = i + self.frames_ahead
            current_window = padded_input_audio[:, window_count-self.frames_ahead:window_count+self.frames_behind+1]
            s = current_window.shape
            current_window = current_window.view((s[0], s[1] * s[2]))
            current_window = torch.unsqueeze(current_window, 1)
            window_audio.append(current_window)
        rtv = torch.cat(window_audio, dim=1)
        return rtv
    def forward(self, input_feature):
        feature_size = int(input_feature.size()[2] / 2)
        mod_audio_self = input_feature[:, :, :feature_size]
        mod_audio_other = input_feature[:, :, feature_size:]
        x1 = self.activation(self.input_layer_self(mod_audio_self))
        x2 = self.activation(self.input_layer_self(mod_audio_other))
        x1_windowed = self.concate_frames(x1)
        x2_windowed = self.concate_frames(x2)
        x_combined = torch.concat([x1_windowed, x2_windowed], axis=2)
        # here I'm assuming that the input_audio is of proper shape
        out, hidden_state = self.lstm(x_combined)
        # bn
        # x = self.bn(out.permute(0, 2, 1)).permute(0, 2, 1)
        x = self.activation(out)
        x = self.output_layer_1(x)
        x = self.output_layer_2(x)
        x = self.output_layer_3(x)
        return x

In [10]:
class SimpleBaselineTransformer_GazePredictionModel(nn.Module):
    def __init__(self, config):
        torch.set_default_tensor_type(torch.DoubleTensor)
        # initialize model
        super(SimpleBaselineTransformer_GazePredictionModel, self).__init__()
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.activation = nn.Sigmoid()
        self.num_layers = config["num_layers"]
        self.config = config
        # the feature of each speaker are encoded with a separate Linear Layer
        self.input_layer_self = nn.Linear(int(config["input_dims"]/2), config["input_layer_out"])
        self.input_layer_other = nn.Linear(int(config["input_dims"]/2), config["input_layer_out"])
        
        # the Recurrent Layer will take care of the next step
        self.lstm_hidden_dims = config["lstm_output_feature_size"]
        self.num_lstm_layer = config["lstm_layer_num"]
        self.frames_ahead = config["frames_ahead"]
        self.frames_behind = config["frames_behind"]
        self.lstm = nn.LSTM(2 * config["input_layer_out"] * (self.frames_ahead + self.frames_behind + 1), 
                            self.lstm_hidden_dims, 
                            self.num_lstm_layer, 
                            batch_first=True)
        self.trasnformer = nn.Transformer(2 * config["input_layer_out"] * (self.frames_ahead + self.frames_behind + 1), batch_first=True,)
        
        # output layers
        self.output_layer_1 = nn.Linear(2 * config["input_layer_out"] * (self.frames_ahead + self.frames_behind + 1), config["output_layer_1_hidden"])
        self.output_layer_1 = nn.Sequential(self.output_layer_1, self.activation, nn.Dropout(self.config["dropout"]))
        self.output_layer_2 = nn.Linear(config["output_layer_1_hidden"], config["output_layer_2_hidden"])
        self.output_layer_2 = nn.Sequential(self.output_layer_2, self.activation, nn.Dropout(self.config["dropout"]))
        self.output_layer_3 = nn.Linear(config["output_layer_2_hidden"], config["output_layer_3_hidden"])
        self.output_layer_3 = nn.Sequential(self.output_layer_3, nn.Sigmoid())

        # audio_filler = torch.tensor([[[-36.04365338911715,0.0,0.0,0.0,0.0,0.0,-3.432169450445466e-14,0.0,0.0,0.0,9.64028691651994e-15,0.0,0.0,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715]]]).to(self.device)
        # text_filler = torch.ones([1, 1, 772]).to(self.device) * -15
        # text_filler[:, :, -4:] = 0
        # self.filler = torch.concat([audio_filler, text_filler], axis=2)
    def concate_frames(self, input_feature):
        # here I expect the 
        padding_front = torch.zeros([input_feature.shape[0], self.frames_ahead, input_feature.shape[2]]).to(self.device)
        padding_back = torch.zeros([input_feature.shape[0], self.frames_behind, input_feature.shape[2]]).to(self.device)
        padded_input_audio = torch.cat([padding_front, input_feature, padding_back], dim=1)
        window_audio = []
        for i in range(0, input_feature.shape[1]):
            window_count = i + self.frames_ahead
            current_window = padded_input_audio[:, window_count-self.frames_ahead:window_count+self.frames_behind+1]
            s = current_window.shape
            current_window = current_window.view((s[0], s[1] * s[2]))
            current_window = torch.unsqueeze(current_window, 1)
            window_audio.append(current_window)
        rtv = torch.cat(window_audio, dim=1)
        return rtv
    def forward(self, input_feature):
        feature_size = int(input_feature.size()[2] / 2)
        mod_audio_self = input_feature[:, :, :feature_size]
        mod_audio_other = input_feature[:, :, feature_size:]
        x1 = self.activation(self.input_layer_self(mod_audio_self))
        x2 = self.activation(self.input_layer_self(mod_audio_other))
        x1_windowed = self.concate_frames(x1)
        x2_windowed = self.concate_frames(x2)
        x_combined = torch.concat([x1_windowed, x2_windowed], axis=2)
        # here I'm assuming that the input_audio is of proper shape
        out, hidden_state = self.lstm(x_combined)
        # bn
        # x = self.bn(out.permute(0, 2, 1)).permute(0, 2, 1)
        x = self.activation(out)
        x = self.output_layer_1(x)
        x = self.output_layer_2(x)
        x = self.output_layer_3(x)
        return x

# Training Loop - For baseline model

In [13]:
def train_model(model, config, train_data, valid_data, wandb, model_name):
    optimiser = torch.optim.Adam(model.parameters(), lr=config["learning_rate"])
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    model.train() 
    loss_fn = nn.CrossEntropyLoss()
    training_loss = []
    valid_loss = []
    training_f1 = []
    valid_f1 = []
    aversion_vs_start = []
    count = 0
    # f1_score = BinaryF1Score(num_classes=2).to(device)
    f1_score = F1Score(task="multiclass", num_classes=2, average="weighted").to(device)
    for epoch in range(1, config['epochs'] + 1):
        total_train_loss = 0
        total_valid_loss = 0
        total_aversion_predicted = 0
        total_train_f1 = 0
        total_valid_f1 = 0
        train_batch_counter = 0
        valid_batch_counter = 0
        total_prediction_counter = 0
        prediction_mean = 0
        prediction_std = 0
        model.zero_grad()
        for _, (X, Y) in enumerate(train_data):
            train_batch_counter += 1
            X, Y = X.to(device), Y.to(device)
            optimiser.zero_grad()
            if "Transformer" in config["model_type"]:
                all_zero = torch.zeros(Y.shape).to(device)
                pred = model(X, all_zero)
            else:Aversion_SelfTap111
                pred = model(X)
            loss = loss_fn(pred.transpose(2, 1), Y.long())
            loss.backward()
            optimiser.step()
            total_train_loss += loss.item()
            # binary_pred = torch.round(pred)
            binary_pred = torch.argmax(pred, axis=2, keepdim=True)
            prediction_mean = torch.mean(binary_pred.float()).item()
            prediction_std = torch.std(binary_pred.float()).item()            
            f1_train = f1_score(binary_pred, torch.unsqueeze(Y, axis=2)).item()
            total_aversion_predicted += torch.sum(binary_pred).item()
            total_prediction_counter += binary_pred.size()[0] * binary_pred.size()[1] 
            total_train_f1 += f1_train
            del X, Y, pred
            torch.cuda.empty_cache()

        total_train_f1 /= train_batch_counter
        total_train_loss /= len(train_data)
        total_aversion_predicted /= total_prediction_counter

        for _, (X, Y) in enumerate(valid_data):
            with torch.no_grad():
                valid_batch_counter += 1
                X, Y = X.to(device), Y.to(device)
                if "Transformer" in config["model_type"]:
                    all_zero = torch.zeros(Y.shape).to(device)
                    pred = model(X, all_zero)
                else:
                    pred = model(X)
                loss = loss_fn(pred.transpose(2, 1), Y.long())
                total_valid_loss += loss.item()

                # binary_pred = torch.round(pred)
                binary_pred = torch.argmax(pred, axis=2, keepdim=True)
                f1_valid = f1_score(binary_pred, torch.unsqueeze(Y, axis=2)).item()
                total_valid_f1 += f1_valid
                del X, Y, pred
                torch.cuda.empty_cache()

        total_valid_f1 /= valid_batch_counter
        total_valid_loss /= len(valid_data)

        if config['wandb']:
            wandb.log({'training loss': total_train_loss,
                        'validation_loss': total_valid_loss,
                        'training_f1': total_train_f1,
                        'validation_f1': total_valid_f1, 
                        "percentage_predicted_aversion": total_aversion_predicted})
        training_loss.append(total_train_loss)
        valid_loss.append(total_valid_loss)
        training_f1.append(total_train_f1)
        valid_f1.append(total_valid_f1)
        aversion_vs_start.append(total_aversion_predicted)
        if total_valid_f1 == max(valid_f1):
            try:
                os.mkdir(os.path.join(*[model_save_location, model_name]))
            except:
                pass
            config_save_path = os.path.join(*[model_save_location, model_name, "config.json"])
            json.dump(config, open(config_save_path, "w"))
            file_name = f'time={datetime.now()}_epoch={epoch}.pt'
            save_path = os.path.join(*[model_save_location, model_name, file_name])
            torch.save(model.state_dict(), save_path)
        if config['early_stopping']>0:
            if epoch > 1:
                if total_valid_f1 < np.mean(valid_f1[epoch - 7:epoch - 2]):
                    count += 1
                else:
                    count = 0
            if count >= config['early_stopping']:
                print('\n\nStopping early due to decrease in performance on validation set\n\n')
                break 
        if count == 0:
            print("Epoch {}, mean: {}, std: {}\ntraining L: {}\nvalidation L:{}".format(epoch, prediction_mean, prediction_std, total_train_f1, total_valid_f1))
        else:
            print("Epoch {}, mean: {}, std: {}\ntraining L: {}\nvalidation L:{}, model have not improved for {} iterations".format(epoch, prediction_mean, prediction_std, total_train_f1, total_valid_f1, count))
    if config['wandb']:
        save_path = os.path.join(*[model_save_location, model_name, file_name])
        wandb.save(save_path)

## Weight and Biases Stuff

## Train the model

In [17]:
config = json.load(open("sentence_config.json", "r"))
# obtain the dataset
torch.set_default_tensor_type(torch.DoubleTensor)
training_dataset = Aversion_SelfTap111(dataset_location, training_set[:10])
validation_dataset = Aversion_SelfTap111(dataset_location, testing_set[:2])
train_dataloader = torch.utils.data.DataLoader(training_dataset, config['batch_size'], True)
valid_dataloader = torch.utils.data.DataLoader(validation_dataset, config['batch_size'], True)
model = SimpleBaseline_GazePredictionModel(config)
train_model(model, config, train_dataloader, valid_dataloader, False, model_save_location)
# run_obj.finish()

RuntimeError: stack expects each tensor to be equal size, but got [300] at entry 0 and [240] at entry 3

# Training Loop - For Audio Only model

In [ ]:
run_obj = wandb.init(project="gaze_prediction", config=config, settings=wandb.Settings(start_method="fork"))

In [16]:
config = json.load(open("sentence_config.json", "r"))
# obtain the dataset
torch.set_default_tensor_type(torch.DoubleTensor)
training_dataset = Aversion_SelfTap111(dataset_location, training_set[:12], audio_only=True)
validation_dataset = Aversion_SelfTap111(dataset_location, testing_set[:2], audio_only=True)
train_dataloader = torch.utils.data.DataLoader(training_dataset, config['batch_size'], True)
valid_dataloader = torch.utils.data.DataLoader(validation_dataset, config['batch_size'], True)
model = SimpleBaseline_GazePredictionModel(config)
train_model(model, config, train_dataloader, valid_dataloader, False, model_save_location)

ValueError: negative dimensions are not allowed

# Training Loop - For sentence level feature Only model (This is actually for word level only)

In [14]:
config = json.load(open("/scratch/ondemand27/evanpan/Gaze_project/training/word_config.json", "r"))
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# obtain the dataset
torch.set_default_tensor_type(torch.DoubleTensor)
training_dataset = Aversion_SelfTap111(dataset_location, training_set, word_timing=True)
validation_dataset = Aversion_SelfTap111(dataset_location, testing_set, word_timing=True)
train_dataloader = torch.utils.data.DataLoader(training_dataset, config['batch_size'], True)
valid_dataloader = torch.utils.data.DataLoader(validation_dataset, config['batch_size'], True)
model = SentenceBaseline_GazePredictionModel(config)
model.to(device)
train_model(model, config, train_dataloader, valid_dataloader, run_obj, model_save_location)

NameError: name 'A' is not defined

## further train the model

In [1]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
config = json.load(open("/scratch/ondemand27/evanpan/Gaze_project/training/word_config.json", "r"))
model = SentenceBaseline_GazePredictionModel(config)
config["learning_rate"] = 0.00001
config["load_model"] = True
if config["wandb"]:
    wandb.login()
    if config["load_model"]:
        run_obj = wandb.init(project="gaze_prediction", config=config, save_code=True,
            resume='allow', id='8w9fyxan')
        # checkpoint_name = "gaze_prediction_team/gaze_prediction/8w9fyxan"
        checkpoint_path = "/scratch/ondemand27/evanpan/data/Gaze_aversion_models/time=2023-04-05 02:37:34.205141_epoch=200.pt"
        wandb.restore(checkpoint_path)
        pretrained_dict = torch.load(checkpoint_path, map_location=device)
        model.load_weights(pretrained_dict)
        model.to(config['device'])
    else:
        run_obj = wandb.init(project="gaze_prediction", config=config, settings=wandb.Settings(start_method="fork"))
else:
    run_obj = None

training_dataset = Aversion_SelfTap111(dataset_location, training_set, sentence_timing=True)
validation_dataset = Aversion_SelfTap111(dataset_location, testing_set, sentence_timing=True)
train_dataloader = torch.utils.data.DataLoader(training_dataset, config['batch_size'], True)
valid_dataloader = torch.utils.data.DataLoader(validation_dataset, config['batch_size'], True)
train_model(model, config, train_dataloader, valid_dataloader, run_obj, model_save_location)

NameError: name 'torch' is not defined

# Training Loop - For sentence level feature Only model (It's real this time)

In [11]:
run_obj = wandb.init(project="gaze_prediction", config=config, settings=wandb.Settings(start_method="fork"))
config = json.load(open("/scratch/ondemand27/evanpan/Gaze_project/training/sentence_config.json", "r"))
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# obtain the dataset
torch.set_default_tensor_type(torch.DoubleTensor)
training_dataset = Aversion_SelfTap111(dataset_location, training_set[0:10], sentence_and_word_timing=True)
validation_dataset = Aversion_SelfTap111(dataset_location, testing_set[0:2], sentence_and_word_timing=True)
train_dataloader = torch.utils.data.DataLoader(training_dataset, config['batch_size'], True)
valid_dataloader = torch.utils.data.DataLoader(validation_dataset, config['batch_size'], True)
model = SentenceBaseline_GazePredictionModel(config)
model.to(device)
train_model(model, config, train_dataloader, valid_dataloader, run_obj, "sentence_and_words")
run_obj.finish()


Epoch 1, mean: 0.17789314687252045, std: 0.3824285864830017
training L: 0.38361384790041525
validation L:0.5202852326801098


/scratch/ondemand27/evanpan/conda_env/.conda/envs/jaligaze/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/scratch/ondemand27/evanpan/conda_env/.conda/envs/jaligaze/lib/python3.10/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Epoch 2, mean: 0.5589313507080078, std: 0.49652254581451416
training L: 0.5135592779060695
validation L:0.5422346910798534
Epoch 3, mean: 0.8715725541114807, std: 0.3345702886581421
training L: 0.5083247797239047
validation L:0.5124457632512047, model have not improved for 1 iterations
Epoch 4, mean: 0.9777710437774658, std: 0.14742977917194366
training L: 0.4620488325757937
validation L:0.5003760834120156, model have not improved for 2 iterations
Epoch 5, mean: 0.9966717958450317, std: 0.05759573355317116
training L: 0.45366407573439577
validation L:0.49920991627236866
Epoch 6, mean: 0.9993893504142761, std: 0.024704914540052414
training L: 0.45135263240753554
validation L:0.49904111077228025
Epoch 7, mean: 0.999908447265625, std: 0.009570656344294548
training L: 0.4510088849744354
validation L:0.4989816254955779, model have not improved for 1 iterations
Epoch 8, mean: 0.9999695420265198, std: 0.005525789689272642
training L: 0.4510375310310284
validation L:0.49904111077228025, model 

wandb: WARNING Saving files without folders. If you want to preserve sub directories pass base_path to wandb.save, i.e. wandb.save("/mnt/folder/file.h5", base_path="/mnt")




Stopping early due to decrease in performance on validation set




percentage_predicted_aversion,▁▄▇██████████████████████████████
training loss,█▅▃▂▂▂▂▂▃▃▃▂▂▂▂▂▁▁▁▂▂▂▂▂▂▁▁▁▁▁▁▁▁
training_f1,▁██▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅
validation_f1,▄█▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▂▂▂▁▁▁▁▁▁▁▁▁
validation_loss,█▅▃▂▂▁▂▂▂▁▂▁▁▂▂▂▂▃▃▃▃▃▃▂▂▂▂▂▁▁▁▁▁
percentage_predicted_aversion,0.99969
training loss,0.67176
training_f1,0.45141
validation_f1,0.49892
validation_loss,0.65346


In [18]:
training_dataset = Aversion_SelfTap111(dataset_location, training_set, sentence_and_word_timing=True)
for i in range(0, len(training_dataset)):
    print(training_dataset[i][0].shape)

(250, 6)
(240, 6)
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/scratch/ondemand27/evanpan/conda_env/.conda/envs/jaligaze/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3460, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_17584/808632026.py", line 3, in <module>
    print(training_dataset[i][0].shape)
  File "/scratch/ondemand27/evanpan/Gaze_project/Dataset_Util/dataloader.py", line 208, in __getitem__
    input_text_on_screen = np.concatenate([input_text_on_screen1, input_text_on_screen2], axis = 1)
  File "<__array_function__ internals>", line 180, in concatenate
ValueError: all the input array dimensions for the concatenation axis must match exactly, but along dimension 0, the array at index 0 has size 250 and the array at index 1 has size 240

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/scratch/ondemand27/evanpan/conda_env/.conda/envs/jaligaze/lib/python3.10/site-package

# Look at the output of the model

In [11]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
config = json.load(open("/scratch/ondemand27/evanpan/Gaze_project/training/sentence_config.json", "r"))
# obtain the dataset
torch.set_default_tensor_type(torch.DoubleTensor)
validation_dataset = Aversion_SelfTap111(dataset_location, testing_set, word_timing=True)
valid_dataloader = torch.utils.data.DataLoader(validation_dataset, config['batch_size'], True)
model = SentenceBaseline_GazePredictionModel(config)
checkpoint_path = "/scratch/ondemand27/evanpan/data/Gaze_aversion_models/time=2023-04-05 02:37:34.205141_epoch=200.pt"
pretrained_dict = torch.load(checkpoint_path, map_location=device)
model.load_state_dict(pretrained_dict)
# train_model(model, config, train_dataloader, valid_dataloader, run_obj, model_save_location)


/usr/bin/nvidia-modprobe: unrecognized option: "-s"

ERROR: Invalid commandline, please run `/usr/bin/nvidia-modprobe --help`
       for usage information.

/usr/bin/nvidia-modprobe: unrecognized option: "-s"

ERROR: Invalid commandline, please run `/usr/bin/nvidia-modprobe --help`
       for usage information.



<All keys matched successfully>

In [10]:
for _, (X, Y) in enumerate(valid_data):
    with torch.no_grad():
        valid_batch_counter += 1
        X, Y = X.to(device), Y.to(device)
        if "Transformer" in config["model_type"]:
            all_zero = torch.zeros(Y.shape).to(device)
            pred = model(X, all_zero)
        else:
            pred = model(X)
        # binary_pred = torch.round(pred)
        binary_pred = torch.argmax(pred, axis=2, keepdim=True)
        f1_valid = f1_score(binary_pred, torch.unsqueeze(Y, axis=2)).item()
        total_valid_f1 += f1_valid
        del X, Y, pred
        torch.cuda.empty_cache()

NameError: name 'valid_data' is not defined

In [32]:
model.to(device)
i = 11
x = torch.from_numpy(validation_dataset[i][0]).to(device)
y = validation_dataset[i][1]
pred = model(torch.unsqueeze(x, axis=0))
pred = torch.softmax(pred, dim=2)
pred = pred.cpu().detach().numpy()
plt.plot(pred[0, :, 0], label="prediction")
plt.plot(y, label="label")
plt.legend()


IndexError: index 2 is out of bounds for dimension 2 with size 2